# Práctica 4 - Lingüística computacional

## Ejercicio 1: algoritmo de Lesk

In [24]:
import nltk
from nltk.corpus import wordnet as wn
from nltk.corpus import stopwords as sw

### Funciones para el algoritmo

In [56]:
# Funcion para calcular el solapamiento entre una frase y una acepcion
def computeOverlap(s, c):
    return len(s.intersection(c))

In [60]:
# Algoritmo de Lesk, decidir el mejor sentido para la palabra segun la frase
# (hace falta eliminar las stopwords de la frase)
def lesk(w, s):
    # inicializar variables
    best_s = "" # mejor significado
    best_o = 0 # mejor solapamiento
    # contexto = la frase sin stopwords
    words = nltk.word_tokenize(s)
    context = set([w for w in words if w not in sw.words('english')])
    # para cada significado, calcular el solapamiento
    for sense in wn.synsets(w):
        definition = set(w.lower() for w in sense.definition().split())
        examples = set(w.lower() for ex in sense.examples() for w in ex.split())
        # defincion y ejemplos del significado
        signature = definition.union(examples)
        # calculo del solapamiento
        overlap = computeOverlap(signature, context)
        # si se encuentra un solapamiento mejor, actualizar datos
        if overlap > best_o:
            best_o = overlap
            best_s = sense.definition()

    # devolver el mejor significado encontrado
    return best_s

### Ejemplo de uso

In [66]:
sentence = "Yesterday I went to the bank to withdraw the money and the credit card did not work"
word = "bank"
significado = lesk(word, sentence)
print("Mejor significado para la palabra '" + word + "':")
if significado:
    print(significado)
else:
    print("!! No se ha encontrado significado")

Mejor significado para la palabra 'bank':
a financial institution that accepts deposits and channels the money into lending activities


## Ejercicio 2: otro algoritmo de desambiguación

In [79]:
import gensim
import nltk
from nltk.data import find
from nltk.corpus import wordnet as wn
from nltk.corpus import stopwords as sw
import numpy as np
from numpy.linalg import norm

nltk.download('word2vec_sample')

[nltk_data] Downloading package word2vec_sample to
[nltk_data]     C:\Users\Aitam\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping models\word2vec_sample.zip.


True

In [68]:
# Funcion para calcular la similitud coseno entre dos vectores
def cos_sim(A, B):
       return np.dot(A, B)/(norm(A)*norm(B))

In [75]:
# Algoritmo similar al de Lesk que utiliza la representacion
# vectorial de las palabras para calcular el mejor significado
def wEmbedding_disambiguation(w, s):
    # cargar el modelo de embedding pre-entrenado del NLTK
    model = gensim.models.KeyedVectors.load_word2vec_format(find("models/word2vec_sample/pruned.word2vec.txt"))
    # inicializar variables
    best_s = "" # mejor significado
    best_sim = 0 # mejor similitud
    # ahora el contexto sera un VECTOR (antes quitar stopwords de la frase)
    sentence = set([w for w in nltk.word_tokenize(s) if w not in sw.words('english')])
    # si la palabra esta en el modelo, obtener su vector
    context_vectors = [model[w] for w in sentence if w in model]
    # calcular el vector del contexto como la media de los vectores de las palabras
    contextV = np.mean(context_vectors, axis=0)  
    # para cada significado, calcular la similitud coseno
    for sense in wn.synsets(w):
        definition = set(w.lower() for w in sense.definition().split())
        examples = set(w.lower() for ex in sense.examples() for w in ex.split())
        # defincion y ejemplos del significado, en forma de VECTOR
        signature = definition.union(examples)
        # si la palabra esta en el modelo, obtener su vector
        signature_vectors = [model[w] for w in signature if w in model]
        # calcular el vector del significado como la media de los vectores de las palabras
        signatureV = np.mean(signature_vectors, axis=0)
        # calculo de la similitud entre el vector del contexto y el vector del significado
        cs = cos_sim(contextV, signatureV)
        # si se encuentra una similitud mejor, actualizar datos
        if cs > best_sim:
            best_sim = cs
            best_s = sense.definition()

    # devolver el mejor significado encontrado
    return best_s

### Ejemplo de uso

In [83]:
sentence = "Yesterday I went to the bank to withdraw the money and the credit card did not work"
word = "bank"
significado = wEmbedding_disambiguation(word, sentence)
print("Mejor significado para la palabra '" + word + "':")
if significado:
    print(significado)
else:
    print("!! No se ha encontrado significado")

Mejor significado para la palabra 'bank':
a financial institution that accepts deposits and channels the money into lending activities
